In [1]:
import os, re, time, json
import pandas as pd
import requests
from requests.adapters import HTTPAdapter, Retry
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Any, Optional
from bs4 import BeautifulSoup

In [ ]:
res = requests.get("https://web.getcha.kr/community/owner-review")

print(res.status_code)

soup = BeautifulSoup(res.text, "html.parser")

# 방법 1: script 태그에서 JSON 데이터 추출
scripts = soup.find_all("script")
print(f"총 script 태그 개수: {len(scripts)}")

# JSON 데이터가 들어있는 script 찾기
for i, script in enumerate(scripts):
    if script.string and "items" in script.string and "id_contents" in script.string:
        print(f"\n{i}번째 script에서 데이터 발견!")

        s = script.string
        start = s.find('{\\\"data\\\"')
        end = s.rfind('}]') +1


        json_str = s[start:end]
        json_str = json_str.replace('\\"', '"')

        data = json.loads(json_str)

data


200
총 script 태그 개수: 90

89번째 script에서 데이터 발견!


{'data': {'page': {'currentPage': 1,
   'totalPage': 228,
   'totalContentsCount': 6837},
  'items': [{'id_contents_migration': 154088,
    'id_forum': 1,
    'type': 11,
    'id_contents': 8870,
    'id_contents_topic': 2,
    'contents_topic': '오너리뷰',
    'topic_tag': None,
    'title': 'EV4',
    'contents': '혼자 출퇴근용으로 타는데 아주 만족입니다! 외관이 조금 이뻤으면 좋겠지만 전비가 좋으니 만족이에요! 트렁크랑 2열높이 개선만 되면 좋을 것 같아요 !! 매니저님께서 알아서 챙겨주시니 좋았습니다 내연',
    'reply_cnt': 2,
    'like_cnt': 3,
    'view_cnt': 107,
    'is_notice': False,
    'is_hot': False,
    'open_yn': True,
    'report_delete_yn': False,
    'extra': None,
    'created': '2025-10-02T13:33:34',
    'user': {'id_user': 1158887,
     'lv': 7,
     'nickname': '너구리5',
     'auth': 'normal',
     'badges': ['https://img.getcha.io/v7_brand_logo/brand_14_dim/badge_kia_dim.png',
      'https://img.getcha.io/v7_brand_logo/brand_14_black/kia.png']},
    'image_url': 'https://img.getcha.io/file/community/live/20251002/1759378942047u2.JPEG#2PcND4JWh4iPhYeTei